
# PrÃ©paration des donnÃ©es pour la dÃ©tection des trouÃ©es forestiÃ¨res par Deep Learning (U-NET et DeepLabV3+)

Ce script permet de prÃ©parer les donnÃ©es pour l'analyse des modÃ¨les
de deep learning dans la dÃ©tection automatique des trouÃ©es forestiÃ¨res Ã  partir
de modÃ¨les numÃ©riques de surface (DSM) et de hauteur de canopÃ©e (CHM).

**Auteur :** VANDER LINDEN Arthur
**Date :** 28-02-2025


# PARTIE 1 : CONFIGURATION

In [9]:
# =====================================================================
# SECTION 1: INITIALISATION ET IMPORTATION DES PACKAGES NÃ‰CESSAIRES
# =====================================================================

# Installation des packages nÃ©cessaires
!pip install -q rasterio==1.3.8 geopandas

# Importation des bibliothÃ¨ques standard
import os
import glob
import pickle
import json
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
import random
import re
from pathlib import Path
from tqdm.notebook import tqdm
from tensorflow.keras.utils import Sequence

# Importation des bibliothÃ¨ques pour le traitement de donnÃ©es gÃ©ospatiales
import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import reproject, Resampling, calculate_default_transform
from datetime import datetime

# Montage de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# =====================================================================
# SECTION 2: CONFIGURATION DES PARAMÃˆTRES ET CHEMINS
# =====================================================================

class Config:
    """Classe de configuration centralisÃ©e pour le projet de dÃ©tection de trouÃ©es forestiÃ¨res"""

    def __init__(self):
        # RÃ©pertoire de base
        self.BASE_DIR = '/content/drive/MyDrive/ForestGaps_DeepLearning_Workflow'

        # Sous-rÃ©pertoires principaux
        self.DATA_DIR = os.path.join(self.BASE_DIR, 'data')
        self.DATA_EXTERNAL_TEST_DIR = os.path.join(self.BASE_DIR, 'data_external_test')
        self.PROCESSED_DIR = os.path.join(self.BASE_DIR, 'processed')
        self.TILES_DIR = os.path.join(self.PROCESSED_DIR, 'tiles')

        # Sous-rÃ©pertoires pour les tuiles
        self.TRAIN_TILES_DIR = os.path.join(self.TILES_DIR, 'train')
        self.VAL_TILES_DIR = os.path.join(self.TILES_DIR, 'val')
        self.TEST_TILES_DIR = os.path.join(self.TILES_DIR, 'test')

        # CrÃ©ation des rÃ©pertoires s'ils n'existent pas
        for dir_path in [
            self.DATA_DIR, self.DATA_EXTERNAL_TEST_DIR, self.PROCESSED_DIR,
            self.TILES_DIR, self.TRAIN_TILES_DIR, self.VAL_TILES_DIR, self.TEST_TILES_DIR
        ]:
            os.makedirs(dir_path, exist_ok=True)

        # ParamÃ¨tres de traitement
        self.TILE_SIZE = 256  # Taille des tuiles en pixels
        self.BATCH_SIZE = 32  # Taille des batchs pour l'entraÃ®nement
        self.THRESHOLDS = [10, 15, 20, 25, 30]  # Seuils de hauteur en mÃ¨tres
        self.TEST_SPLIT = 0.15  # Proportion de donnÃ©es pour le test
        self.VAL_SPLIT = 0.15   # Proportion de donnÃ©es pour la validation
        self.OVERLAP = 0.30  # Chevauchement entre les tuiles lors de l'extraction

    def save_config(self, filepath=None):
        """Sauvegarde la configuration actuelle dans un fichier JSON"""
        if filepath is None:
            filepath = os.path.join(self.PROCESSED_DIR, 'config.json')

        # CrÃ©ation d'un dictionnaire de configuration
        config_dict = {k: v for k, v in self.__dict__.items() if not k.startswith('__') and not callable(v)}

        # Conversion des chemins en chaÃ®nes de caractÃ¨res
        for k, v in config_dict.items():
            if isinstance(v, Path):
                config_dict[k] = str(v)

        # Sauvegarde dans un fichier JSON
        with open(filepath, 'w') as f:
            json.dump(config_dict, f, indent=4)

        print(f"Configuration sauvegardÃ©e dans {filepath}")

    def load_config(self, filepath):
        """Charge une configuration Ã  partir d'un fichier JSON"""
        with open(filepath, 'r') as f:
            config_dict = json.load(f)

        # Mise Ã  jour des attributs
        for k, v in config_dict.items():
            setattr(self, k, v)

        print(f"Configuration chargÃ©e depuis {filepath}")

# Initialisation de la configuration
config = Config()

In [11]:
# =====================================================================
# SECTION 3: FONCTIONS UTILITAIRES POUR LE TRAITEMENT DES DONNÃ‰ES
# =====================================================================

def find_chm_dsm_pairs(directory):
    """
    Trouve toutes les paires CHM/DSM dans un rÃ©pertoire.
    Les paires sont identifiÃ©es par leur prÃ©fixe commun avant les suffixes _CHM.tif et _DSM.tif.

    Args:
        directory: RÃ©pertoire contenant les donnÃ©es

    Returns:
        Une liste de tuples (chemin_chm, chemin_dsm, prÃ©fixe)
    """
    # Rechercher tous les fichiers CHM et DSM
    chm_files = glob.glob(os.path.join(directory, '*CHM.tif'))
    dsm_files = glob.glob(os.path.join(directory, '*DSM.tif'))

    # Dictionnaires pour stocker les chemins par prÃ©fixe
    chm_dict = {}
    dsm_dict = {}

    # Extraire les prÃ©fixes des fichiers CHM
    for chm_path in chm_files:
        filename = os.path.basename(chm_path)
        prefix = filename.replace('CHM.tif', '').rstrip('_')
        chm_dict[prefix] = chm_path

    # Extraire les prÃ©fixes des fichiers DSM
    for dsm_path in dsm_files:
        filename = os.path.basename(dsm_path)
        prefix = filename.replace('DSM.tif', '').rstrip('_')
        dsm_dict[prefix] = dsm_path

    # Trouver les prÃ©fixes communs (les paires)
    pairs = []
    for prefix in chm_dict:
        if prefix in dsm_dict:
            pairs.append((chm_dict[prefix], dsm_dict[prefix], prefix))

    # Trier les paires par prÃ©fixe pour une sortie cohÃ©rente
    return sorted(pairs, key=lambda x: x[2])

def analyze_raster_pair(dsm_path, chm_path, prefix):
    """
    Analyse une paire de rasters DSM/CHM et gÃ©nÃ¨re un rapport dÃ©taillÃ©.

    Args:
        dsm_path: Chemin vers le fichier DSM
        chm_path: Chemin vers le fichier CHM
        prefix: PrÃ©fixe du site

    Returns:
        Dictionnaire contenant le rapport d'analyse
    """
    report = {
        'site_id': prefix,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'valid': True,
        'warnings': [],
        'errors': [],
        'metadata': {'dsm': {}, 'chm': {}},
        'statistics': {'dsm': {}, 'chm': {}},
        'alignment_status': 'aligned',
        'processing_needed': [],
        'processing_history': []
    }

    try:
        # Analyse du DSM
        with rasterio.open(dsm_path) as dsm:
            dsm_data = dsm.read(1)

            # MÃ©tadonnÃ©es
            report['metadata']['dsm'] = {
                'path': dsm_path,
                'shape': (dsm.width, dsm.height),
                'resolution': dsm.res,
                'crs': str(dsm.crs),
                'bounds': {
                    'left': dsm.bounds.left,
                    'bottom': dsm.bounds.bottom,
                    'right': dsm.bounds.right,
                    'top': dsm.bounds.top
                },
                'transform': list(dsm.transform),
                'nodata': dsm.nodata,
                'driver': dsm.driver
            }

            # Statistiques basiques
            valid_mask = ~np.isnan(dsm_data)
            if dsm.nodata is not None:
                valid_mask &= (dsm_data != dsm.nodata)

            if np.any(valid_mask):
                report['statistics']['dsm'] = {
                    'min': float(np.min(dsm_data[valid_mask])),
                    'max': float(np.max(dsm_data[valid_mask])),
                    'mean': float(np.mean(dsm_data[valid_mask])),
                    'std': float(np.std(dsm_data[valid_mask])),
                    'valid_ratio': float(np.sum(valid_mask) / dsm_data.size)
                }
            else:
                report['statistics']['dsm'] = {
                    'valid_ratio': 0.0
                }
                report['errors'].append("Le DSM ne contient aucune donnÃ©e valide")
                report['valid'] = False

        # Analyse du CHM
        with rasterio.open(chm_path) as chm:
            chm_data = chm.read(1)

            # MÃ©tadonnÃ©es
            report['metadata']['chm'] = {
                'path': chm_path,
                'shape': (chm.width, chm.height),
                'resolution': chm.res,
                'crs': str(chm.crs),
                'bounds': {
                    'left': chm.bounds.left,
                    'bottom': chm.bounds.bottom,
                    'right': chm.bounds.right,
                    'top': chm.bounds.top
                },
                'transform': list(chm.transform),
                'nodata': chm.nodata,
                'driver': chm.driver
            }

            # Statistiques basiques
            valid_mask = ~np.isnan(chm_data)
            if chm.nodata is not None:
                valid_mask &= (chm_data != chm.nodata)

            if np.any(valid_mask):
                report['statistics']['chm'] = {
                    'min': float(np.min(chm_data[valid_mask])),
                    'max': float(np.max(chm_data[valid_mask])),
                    'mean': float(np.mean(chm_data[valid_mask])),
                    'std': float(np.std(chm_data[valid_mask])),
                    'valid_ratio': float(np.sum(valid_mask) / chm_data.size)
                }
            else:
                report['statistics']['chm'] = {
                    'valid_ratio': 0.0
                }
                report['errors'].append("Le CHM ne contient aucune donnÃ©e valide")
                report['valid'] = False

        # VÃ©rification des projections
        with rasterio.open(dsm_path) as dsm, rasterio.open(chm_path) as chm:
            if dsm.crs != chm.crs:
                report['warnings'].append(f"Projections diffÃ©rentes: DSM={dsm.crs}, CHM={chm.crs}")
                report['alignment_status'] = 'misaligned'
                report['processing_needed'].append('reproject')

            # VÃ©rifier les rÃ©solutions
            resolution_threshold = 0.01  # TolÃ©rance pour les diffÃ©rences de rÃ©solution
            if (abs(dsm.res[0] - chm.res[0]) > resolution_threshold or
                abs(dsm.res[1] - chm.res[1]) > resolution_threshold):
                report['warnings'].append(f"RÃ©solutions diffÃ©rentes: DSM={dsm.res}, CHM={chm.res}")
                report['alignment_status'] = 'misaligned'
                report['processing_needed'].append('resample')

            # VÃ©rifier les emprises
            bounds_threshold = 1.0  # TolÃ©rance pour les diffÃ©rences d'emprise en unitÃ©s de carte
            if (abs(dsm.bounds.left - chm.bounds.left) > bounds_threshold or
                abs(dsm.bounds.bottom - chm.bounds.bottom) > bounds_threshold or
                abs(dsm.bounds.right - chm.bounds.right) > bounds_threshold or
                abs(dsm.bounds.top - chm.bounds.top) > bounds_threshold):
                report['warnings'].append("Emprises diffÃ©rentes")
                report['alignment_status'] = 'misaligned'
                report['processing_needed'].append('align_extent')

            # VÃ©rifier des dimensions en pixels
            if dsm.width != chm.width or dsm.height != chm.height:
                report['warnings'].append(f"Dimensions diffÃ©rentes: DSM={dsm.width}x{dsm.height}, CHM={chm.width}x{chm.height}")

    except Exception as e:
        report['valid'] = False
        report['errors'].append(f"Erreur lors de l'analyse: {str(e)}")

    return report

def align_rasters(dsm_path, chm_path, report, config):
    """
    Aligne les rasters DSM et CHM pour qu'ils aient la mÃªme projection, rÃ©solution et emprise.
    Utilise le DSM comme rÃ©fÃ©rence pour aligner le CHM.

    Args:
        dsm_path: Chemin vers le fichier DSM
        chm_path: Chemin vers le fichier CHM
        report: Rapport d'analyse
        config: Configuration du projet

    Returns:
        Tuple (dsm_path_aligned, chm_path_aligned, updated_report)
    """
    # Si les rasters sont dÃ©jÃ  alignÃ©s, rien Ã  faire
    if report['alignment_status'] == 'aligned':
        return dsm_path, chm_path, report

    # CrÃ©er un dossier pour les fichiers traitÃ©s
    aligned_dir = os.path.join(config.PROCESSED_DIR, 'aligned_rasters')
    os.makedirs(aligned_dir, exist_ok=True)

    prefix = report['site_id']

    # Chemins des fichiers de sortie
    dsm_aligned_path = dsm_path  # Par dÃ©faut, pas de modification
    chm_aligned_path = os.path.join(aligned_dir, f"{prefix}_CHM_aligned.tif")

    # Ouvrir les fichiers source
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(chm_path) as chm_src:
        # Utiliser le DSM comme rÃ©fÃ©rence pour le nouveau CHM
        dst_crs = dsm_src.crs
        dst_transform = dsm_src.transform
        dst_width = dsm_src.width
        dst_height = dsm_src.height
        dst_nodata = chm_src.nodata if chm_src.nodata is not None else 0

        # Si les projections diffÃ¨rent, recalculer la transformation
        if 'reproject' in report['processing_needed'] or 'resample' in report['processing_needed']:
            dst_transform, dst_width, dst_height = calculate_default_transform(
                chm_src.crs, dst_crs, dsm_src.width, dsm_src.height,
                *dsm_src.bounds
            )

        # PrÃ©parer les mÃ©tadonnÃ©es pour le CHM alignÃ©
        dst_profile = chm_src.profile.copy()
        dst_profile.update({
            'crs': dst_crs,
            'transform': dst_transform,
            'width': dst_width,
            'height': dst_height,
            'nodata': dst_nodata
        })

        # CrÃ©er le fichier CHM alignÃ©
        with rasterio.open(chm_aligned_path, 'w', **dst_profile) as dst:
            # Reprojeter/RÃ©echantillonner le CHM
            reproject(
                source=rasterio.band(chm_src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=chm_src.transform,
                src_crs=chm_src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                src_nodata=chm_src.nodata,
                dst_nodata=dst_nodata,
                resampling=Resampling.bilinear
            )

        # Mettre Ã  jour le rapport
        action = "Alignement du CHM avec le DSM: "
        if 'reproject' in report['processing_needed']:
            action += f"reprojection de {chm_src.crs} vers {dst_crs}, "
        if 'resample' in report['processing_needed']:
            action += f"rÃ©Ã©chantillonnage de {chm_src.res} vers {dsm_src.res}, "
        if 'align_extent' in report['processing_needed']:
            action += "alignement des emprises, "

        report['processing_history'].append(action.rstrip(", "))
        report['alignment_status'] = 'aligned'
        report['warnings'].append(f"Le CHM a Ã©tÃ© alignÃ© sur le DSM: {chm_aligned_path}")

    return dsm_aligned_path, chm_aligned_path, report

def process_raster_pair_robustly(dsm_path, chm_path, prefix, config, save_report=True, force_reanalysis=False):
    """
    Traite une paire de rasters DSM/CHM de maniÃ¨re robuste.
    Ã‰vite de rÃ©pÃ©ter l'analyse si elle a dÃ©jÃ  Ã©tÃ© effectuÃ©e.

    Args:
        dsm_path: Chemin vers le fichier DSM
        chm_path: Chemin vers le fichier CHM
        prefix: PrÃ©fixe du site
        config: Configuration du projet
        save_report: Si True, sauvegarde le rapport au format JSON
        force_reanalysis: Si True, force la rÃ©analyse mÃªme si un rapport existe dÃ©jÃ 

    Returns:
        Tuple (dsm_path_processed, chm_path_processed, report, success)
    """
    print(f"Analyse de la paire: {prefix}")

    # VÃ©rifier si l'analyse a dÃ©jÃ  Ã©tÃ© effectuÃ©e
    report_dir = os.path.join(config.PROCESSED_DIR, 'reports')
    os.makedirs(report_dir, exist_ok=True)
    report_path = os.path.join(report_dir, f"{prefix}_report.json")

    if not force_reanalysis and os.path.exists(report_path):
        try:
            with open(report_path, 'r') as f:
                report = json.load(f)

            print(f"Utilisation de l'analyse existante pour {prefix}")

            # RÃ©cupÃ©rer les chemins traitÃ©s
            dsm_processed = dsm_path  # Par dÃ©faut, utiliser le chemin d'origine

            # VÃ©rifier si le rapport contient un chemin alignÃ© pour le CHM
            if 'alignment_status' in report and report['alignment_status'] == 'aligned':
                # Si c'Ã©tait dÃ©jÃ  alignÃ©, utiliser le chemin d'origine
                if not report['processing_history']:
                    chm_processed = chm_path
                else:
                    # Sinon, reconstruire le chemin du CHM alignÃ©
                    aligned_dir = os.path.join(config.PROCESSED_DIR, 'aligned_rasters')
                    chm_processed = os.path.join(aligned_dir, f"{prefix}_CHM_aligned.tif")

                    # VÃ©rifier que le fichier existe bien
                    if not os.path.exists(chm_processed):
                        print(f"âš ï¸ Fichier CHM alignÃ© manquant: {chm_processed}")
                        print("RÃ©analyse de la paire...")
                        return process_raster_pair_robustly(dsm_path, chm_path, prefix, config, save_report, True)
            else:
                # Si pas alignÃ©, utiliser le chemin d'origine
                chm_processed = chm_path

            return dsm_processed, chm_processed, report, True

        except Exception as e:
            print(f"âš ï¸ Erreur lors de la lecture du rapport existant: {str(e)}")
            print("RÃ©analyse de la paire...")

    # Si pas de rapport existant ou force_reanalysis, faire l'analyse complÃ¨te
    # GÃ©nÃ©rer le rapport d'analyse
    report = analyze_raster_pair(dsm_path, chm_path, prefix)

    # VÃ©rifier s'il y a des erreurs critiques
    if not report['valid']:
        print(f"ERREUR: Paire invalide ({prefix})")
        for error in report['errors']:
            print(f"  - {error}")

        if save_report:
            error_report_path = os.path.join(report_dir, f"{prefix}_error_report.json")
            with open(error_report_path, 'w') as f:
                json.dump(report, f, indent=2)

        return dsm_path, chm_path, report, False

    # Afficher un rÃ©sumÃ© du rapport
    print(f"  DSM: {report['metadata']['dsm']['shape'][0]}x{report['metadata']['dsm']['shape'][1]}, rÃ©solution: {report['metadata']['dsm']['resolution']}")
    print(f"  CHM: {report['metadata']['chm']['shape'][0]}x{report['metadata']['chm']['shape'][1]}, rÃ©solution: {report['metadata']['chm']['resolution']}")

    if report['alignment_status'] == 'misaligned':
        print(f"  Alignement requis: {', '.join(report['processing_needed'])}")

        # Aligner les rasters si nÃ©cessaire
        dsm_path, chm_path, report = align_rasters(dsm_path, chm_path, report, config)
        print(f"  â†’ Alignement effectuÃ©")
    else:
        print(f"  âœ“ Alignement correct")

    # Sauvegarder le rapport
    if save_report:
        with open(report_path, 'w') as f:
            json.dump(report, f, indent=2)

    return dsm_path, chm_path, report, True

def create_gap_masks(chm_path, thresholds, output_dir, prefix, force_regenerate=False):
    """
    CrÃ©e des masques binaires de trouÃ©es Ã  partir d'un CHM pour diffÃ©rents seuils de hauteur.
    VÃ©rifie physiquement si chaque masque existe avant de le crÃ©er.

    Args:
        chm_path: Chemin vers le fichier CHM
        thresholds: Liste des seuils de hauteur (en mÃ¨tres)
        output_dir: RÃ©pertoire de sortie pour les masques
        prefix: PrÃ©fixe du site pour nommer les fichiers
        force_regenerate: Si True, force la rÃ©gÃ©nÃ©ration des masques mÃªme s'ils existent dÃ©jÃ 

    Returns:
        Liste des chemins vers les masques crÃ©Ã©s
    """
    os.makedirs(output_dir, exist_ok=True)
    mask_paths = {}
    all_masks_exist = True

    # VÃ©rifier pour chaque seuil
    for threshold in thresholds:
        output_path = os.path.join(output_dir, f'{prefix}_gap_mask_{threshold}m.tif')
        mask_paths[threshold] = output_path

        # VÃ©rifier si le fichier existe rÃ©ellement
        if os.path.isfile(output_path) and not force_regenerate:
            try:
                # VÃ©rifier que le fichier est valide en l'ouvrant
                with rasterio.open(output_path) as src:
                    # Lire un petit Ã©chantillon pour vÃ©rifier l'intÃ©gritÃ©
                    sample = src.read(1, window=Window(0, 0, 10, 10))
                print(f"Masque pour le seuil {threshold}m - site {prefix} dÃ©jÃ  existant. Utilisation du fichier existant.")
                continue
            except Exception as e:
                print(f"âš ï¸ Masque existant corrompu pour le seuil {threshold}m - site {prefix}: {str(e)}")
                all_masks_exist = False
        else:
            all_masks_exist = False

    # Si tous les masques existent et sont valides, retourner les chemins
    if all_masks_exist:
        return mask_paths

    # Pour les masques manquants ou corrompus, crÃ©er les masques
    for threshold in thresholds:
        output_path = mask_paths[threshold]
        if os.path.isfile(output_path) and not force_regenerate:
            continue  # Sauter ceux qui existent dÃ©jÃ  et sont valides

        print(f"CrÃ©ation du masque pour le seuil {threshold}m - site {prefix}...")

        # Ouvrir le CHM (seulement si nÃ©cessaire)
        with rasterio.open(chm_path) as src:
            chm_data = src.read(1)
            meta = src.meta.copy()

        # Modifier les mÃ©tadonnÃ©es pour les masques binaires
        meta.update(dtype='uint8', nodata=255)

        # CrÃ©er un masque binaire (1=trouÃ©e, 0=non-trouÃ©e)
        gap_mask = np.zeros_like(chm_data, dtype='uint8')

        # Les pixels avec hauteur < seuil sont des trouÃ©es (1)
        mask_valid = ~np.isnan(chm_data)
        gap_mask[mask_valid & (chm_data < threshold)] = 1
        gap_mask[~mask_valid] = 255  # nodata value

        # Sauvegarder le masque
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(gap_mask, 1)

        # Calcul des statistiques
        valid_pixels = np.sum(mask_valid)
        gap_pixels = np.sum(gap_mask[mask_valid] == 1)
        gap_pct = gap_pixels / valid_pixels * 100 if valid_pixels > 0 else 0

        print(f"  Total pixels valides: {valid_pixels}")
        print(f"  Pixels de trouÃ©es: {gap_pixels} ({gap_pct:.2f}%)")

    return mask_paths

def calculate_and_save_gap_statistics(gap_masks_by_prefix, config):
    """
    Calcule et sauvegarde les statistiques globales de trouÃ©es par seuil.
    Convertit les types NumPy en types Python natifs pour Ã©viter l'erreur de sÃ©rialisation JSON.

    Args:
        gap_masks_by_prefix: Dictionnaire des chemins de masques par prÃ©fixe et par seuil
        config: Configuration du projet

    Returns:
        Dictionnaire des ratios de trouÃ©es par seuil
    """
    # Initialiser les compteurs
    total_pixels_by_threshold = {threshold: 0 for threshold in config.THRESHOLDS}
    gap_pixels_by_threshold = {threshold: 0 for threshold in config.THRESHOLDS}

    # Parcourir tous les sites
    for prefix, mask_paths in gap_masks_by_prefix.items():
        for threshold, mask_path in mask_paths.items():
            try:
                with rasterio.open(mask_path) as src:
                    mask_data = src.read(1)
                    valid_mask = (mask_data != 255)
                    total_valid = np.sum(valid_mask)
                    gap_pixels = np.sum(mask_data[valid_mask] == 1)

                    # Convertir les types NumPy en types Python natifs
                    total_pixels_by_threshold[threshold] += int(total_valid)
                    gap_pixels_by_threshold[threshold] += int(gap_pixels)
            except Exception as e:
                print(f"âš ï¸ Erreur lors de l'analyse du masque {mask_path}: {str(e)}")
                continue

    # Calculer les ratios
    gap_ratios = {}
    for threshold in config.THRESHOLDS:
        if total_pixels_by_threshold[threshold] > 0:
            gap_ratios[threshold] = float(gap_pixels_by_threshold[threshold]) / float(total_pixels_by_threshold[threshold])
        else:
            gap_ratios[threshold] = 0.0  # Valeur par dÃ©faut si aucun pixel valide

    # PrÃ©parer les donnÃ©es Ã  sauvegarder
    stats_data = {
        'total_pixels': {str(k): int(v) for k, v in total_pixels_by_threshold.items()},
        'gap_pixels': {str(k): int(v) for k, v in gap_pixels_by_threshold.items()},
        'gap_ratios': {str(k): float(v) for k, v in gap_ratios.items()},
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'prefixes': list(gap_masks_by_prefix.keys())  # Liste des prÃ©fixes inclus dans ces statistiques
    }

    # Sauvegarde sÃ©curisÃ©e: Ã©crire d'abord dans un fichier temporaire
    stats_path = os.path.join(config.PROCESSED_DIR, 'gap_statistics.json')
    temp_path = stats_path + ".tmp"

    try:
        # Ã‰crire dans un fichier temporaire
        with open(temp_path, 'w') as f:
            json.dump(stats_data, f, indent=2)

        # Si l'Ã©criture a rÃ©ussi, renommer le fichier temporaire
        os.replace(temp_path, stats_path)

        # CrÃ©er Ã©galement un fichier de backup
        backup_path = stats_path + ".backup"
        with open(backup_path, 'w') as f:
            json.dump(stats_data, f, indent=2)

    except Exception as e:
        print(f"âš ï¸ Erreur lors de la sauvegarde des statistiques: {str(e)}")
        # Supprimer le fichier temporaire si l'opÃ©ration a Ã©chouÃ©
        if os.path.exists(temp_path):
            os.remove(temp_path)

    print("\nStatistiques globales des trouÃ©es:")
    for threshold, ratio in gap_ratios.items():
        print(f"  Seuil {threshold}m: {ratio:.2%} de pixels sont des trouÃ©es")

    return gap_ratios

def safe_load_json(file_path, backup_suffix="_backup"):
    """
    Charge un fichier JSON de maniÃ¨re sÃ©curisÃ©e, avec gestion des erreurs et fichier de backup.

    Args:
        file_path: Chemin vers le fichier JSON Ã  charger
        backup_suffix: Suffixe pour le fichier de backup

    Returns:
        Dictionnaire chargÃ© ou None si Ã©chec
    """
    if not os.path.exists(file_path):
        return None

    # Essayer de charger le fichier principal
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except Exception as main_error:
        print(f"âš ï¸ Erreur lors du chargement de {file_path}: {str(main_error)}")

        # Essayer de charger le backup si disponible
        backup_path = file_path + backup_suffix
        if os.path.exists(backup_path):
            try:
                print(f"Tentative de restauration depuis le backup...")
                with open(backup_path, 'r') as f:
                    data = json.load(f)

                # Si le backup fonctionne, l'utiliser pour rÃ©parer le fichier principal
                with open(file_path, 'w') as f:
                    json.dump(data, f, indent=2)
                print(f"âœ… Restauration rÃ©ussie depuis le backup.")

                return data
            except Exception as backup_error:
                print(f"âŒ Ã‰chec de la restauration depuis le backup: {str(backup_error)}")

        return None

def split_sites_by_percentage(data_pairs, train_percent=0.7, val_percent=0.15, test_percent=0.15):
    """
    Divise les sites en ensembles d'entraÃ®nement, validation et test
    selon les pourcentages spÃ©cifiÃ©s.

    Args:
        data_pairs: Liste des tuples (chm_path, dsm_path, prefix)
        train_percent, val_percent, test_percent: Pourcentages pour chaque ensemble

    Returns:
        Dictionnaire contenant les ensembles d'entraÃ®nement, validation et test
    """
    # VÃ©rification des pourcentages
    assert abs(train_percent + val_percent + test_percent - 1.0) < 1e-10, "Les pourcentages doivent faire 100%"

    # MÃ©langer les donnÃ©es pour une division alÃ©atoire
    np.random.seed(42)  # Pour la reproductibilitÃ©
    indices = np.random.permutation(len(data_pairs))

    # Calculer les indices de sÃ©paration
    train_end = int(train_percent * len(data_pairs))
    val_end = train_end + int(val_percent * len(data_pairs))

    # Diviser les donnÃ©es
    train_indices = indices[:train_end]
    val_indices = indices[train_end:val_end]
    test_indices = indices[val_end:]

    # CrÃ©er les sous-ensembles
    train_pairs = [data_pairs[i] for i in train_indices]
    val_pairs = [data_pairs[i] for i in val_indices]
    test_pairs = [data_pairs[i] for i in test_indices]

    return {
        'train': train_pairs,
        'val': val_pairs,
        'test': test_pairs
    }

def update_site_split(existing_split, all_data_pairs, train_percent=0.7, val_percent=0.15, test_percent=0.15):
    """
    Met Ã  jour la division existante avec de nouveaux sites tout en respectant les proportions.

    Args:
        existing_split: Dictionnaire de la division existante
        all_data_pairs: Liste de tous les tuples (chm_path, dsm_path, prefix) actuels
        config: Instance de la classe Config

    Returns:
        Dictionnaire mis Ã  jour avec la nouvelle division
    """
    # Extraire les prÃ©fixes des sites existants
    existing_prefixes = set()
    for subset in ['train', 'val', 'test']:
        existing_prefixes.update(p[2] for p in existing_split[subset])

    # Identifier les nouveaux sites
    new_sites = [pair for pair in all_data_pairs if pair[2] not in existing_prefixes]

    if not new_sites:
        print("Aucun nouveau site Ã  ajouter Ã  la division existante.")
        return existing_split

    print(f"Ajout de {len(new_sites)} nouveaux sites Ã  la division existante:")
    for pair in new_sites:
        print(f"  - {pair[2]}")

    # Calculer les nombres actuels dans chaque catÃ©gorie
    current_counts = {subset: len(existing_split[subset]) for subset in ['train', 'val', 'test']}
    current_total = sum(current_counts.values())

    # Calculer les nombres cibles aprÃ¨s ajout des nouveaux sites
    new_total = current_total + len(new_sites)
    target_counts = {
        'train': int(new_total * train_percent),
        'val': int(new_total * val_percent),
        'test': new_total - int(new_total * train_percent) - int(new_total * val_percent)
    }

    # Calculer combien de sites doivent Ãªtre ajoutÃ©s Ã  chaque catÃ©gorie
    to_add = {
        subset: max(0, target_counts[subset] - current_counts[subset])
        for subset in ['train', 'val', 'test']
    }

    print("RÃ©partition cible:")
    print(f"  Train: {target_counts['train']} sites ({train_percent*100:.1f}%)")
    print(f"  Validation: {target_counts['val']} sites ({val_percent*100:.1f}%)")
    print(f"  Test: {target_counts['test']} sites ({test_percent*100:.1f}%)")

    print("Nombre de nouveaux sites Ã  ajouter dans chaque ensemble:")
    print(f"  Train: +{to_add['train']} sites")
    print(f"  Validation: +{to_add['val']} sites")
    print(f"  Test: +{to_add['test']} sites")

    # MÃ©langer les nouveaux sites pour une rÃ©partition alÃ©atoire
    np.random.seed(42)  # Pour la reproductibilitÃ©
    np.random.shuffle(new_sites)

    # RÃ©partir les nouveaux sites
    added_count = 0
    for subset in ['train', 'val', 'test']:
        if added_count >= len(new_sites):
            break

        # Prendre le nombre requis de sites pour cette catÃ©gorie
        sites_to_add = new_sites[added_count:added_count + to_add[subset]]
        existing_split[subset].extend(sites_to_add)
        added_count += len(sites_to_add)

    # Si des sites restent (Ã  cause des arrondis), les ajouter Ã  l'ensemble d'entraÃ®nement
    if added_count < len(new_sites):
        remaining_sites = new_sites[added_count:]
        print(f"  {len(remaining_sites)} sites supplÃ©mentaires ajoutÃ©s Ã  l'ensemble d'entraÃ®nement (pour Ã©quilibrage)")
        existing_split['train'].extend(remaining_sites)

    # Afficher la rÃ©partition finale
    print("\nRÃ©partition finale:")
    print(f"  Train: {len(existing_split['train'])} sites ({len(existing_split['train'])/new_total*100:.1f}%)")
    print(f"  Validation: {len(existing_split['val'])} sites ({len(existing_split['val'])/new_total*100:.1f}%)")
    print(f"  Test: {len(existing_split['test'])} sites ({len(existing_split['test'])/new_total*100:.1f}%)")

    return existing_split

def update_tile_info(existing_info, site_split, subset_name, output_dir, config):
    """
    Met Ã  jour les informations de tuiles avec les nouveaux sites.

    Args:
        existing_info: Liste des informations de tuiles existantes
        site_split: Dictionnaire de la division des sites
        subset_name: Nom du sous-ensemble ('train', 'val', ou 'test')
        output_dir: RÃ©pertoire de sortie pour les tuiles
        config: Configuration du projet

    Returns:
        Liste mise Ã  jour des informations de tuiles
    """
    # Extraire les prÃ©fixes des sites dÃ©jÃ  traitÃ©s
    existing_prefixes = set(info['site'] for info in existing_info)

    # Identifier les sites Ã  traiter
    sites_to_process = [pair for pair in site_split[subset_name]
                        if pair[2] not in existing_prefixes]

    if not sites_to_process:
        print(f"Aucun nouveau site Ã  traiter pour l'ensemble {subset_name}.")
        return existing_info

    print(f"\nExtraction des tuiles pour {len(sites_to_process)} nouveaux sites dans l'ensemble {subset_name}:")
    for pair in sites_to_process:
        print(f"  - {pair[2]}")

    # Traiter chaque nouveau site
    new_info = []
    gap_masks_dir = os.path.join(config.PROCESSED_DIR, 'gap_masks')

    for chm_path, dsm_path, prefix in sites_to_process:
        # Construire les chemins vers les masques
        mask_paths = {
            threshold: os.path.join(gap_masks_dir, f'{prefix}_gap_mask_{threshold}m.tif')
            for threshold in config.THRESHOLDS
        }

        # Extraire les tuiles pour ce site
        site_info = create_dataset_from_site(
            dsm_path, mask_paths, config.THRESHOLDS, config.TILE_SIZE,
            overlap=config.OVERLAP, output_dir=output_dir, site_prefix=prefix
        )

        new_info.extend(site_info)

    # Combiner les informations existantes et nouvelles
    combined_info = existing_info + new_info
    print(f"Nombre total de tuiles pour {subset_name}: {len(combined_info)} "
          f"({len(existing_info)} existantes + {len(new_info)} nouvelles)")

    # Si c'est l'ensemble d'entraÃ®nement, mettre Ã  jour les indices Ã©quilibrÃ©s
    if subset_name == 'train' and len(new_info) > 0:
        # Charger les ratios de trouÃ©es
        gap_ratios_path = os.path.join(config.PROCESSED_DIR, 'gap_ratios.json')
        if os.path.exists(gap_ratios_path):
            try:
                with open(gap_ratios_path, 'r') as f:
                    gap_ratios_data = json.load(f)
                    gap_ratios = {int(k): float(v) for k, v in gap_ratios_data.items()}

                # Charger les indices existants s'ils existent
                train_indices_path = os.path.join(config.PROCESSED_DIR, 'balanced_train_indices.pkl')
                if os.path.exists(train_indices_path):
                    try:
                        with open(train_indices_path, 'rb') as f:
                            existing_indices = pickle.load(f)

                        # Mettre Ã  jour les indices avec les nouvelles tuiles
                        updated_indices = update_balanced_indices(
                            existing_indices, new_info, existing_info,
                            config.THRESHOLDS, gap_ratios
                        )

                        # Sauvegarder les indices mis Ã  jour
                        with open(train_indices_path, 'wb') as f:
                            pickle.dump(updated_indices, f)

                        print(f"Indices Ã©quilibrÃ©s mis Ã  jour: {len(updated_indices)} entrÃ©es")
                    except Exception as e:
                        print(f"âš ï¸ Erreur lors de la mise Ã  jour des indices Ã©quilibrÃ©s: {str(e)}")
                        import traceback
                        traceback.print_exc()
                else:
                    # Si pas d'indices existants mais nouvelles tuiles, crÃ©er des indices
                    print("CrÃ©ation de nouveaux indices Ã©quilibrÃ©s...")
                    all_indices = create_balanced_indices(combined_info, config.THRESHOLDS, gap_ratios)

                    # Sauvegarder les indices
                    with open(train_indices_path, 'wb') as f:
                        pickle.dump(all_indices, f)

                    print(f"Nouveaux indices Ã©quilibrÃ©s crÃ©Ã©s: {len(all_indices)} entrÃ©es")
            except Exception as e:
                print(f"âš ï¸ Erreur lors de la crÃ©ation/mise Ã  jour des indices Ã©quilibrÃ©s: {str(e)}")
                import traceback
                traceback.print_exc()

    return combined_info

def verify_tiles_integrity(tile_info):
    """
    VÃ©rifie si toutes les tuiles rÃ©fÃ©rencÃ©es dans les mÃ©tadonnÃ©es existent physiquement.

    Args:
        tile_info: Liste d'informations sur les tuiles

    Returns:
        Tuple (valide, tuiles_valides, total_tuiles)
    """
    missing = 0
    total = len(tile_info)
    valid_info = []

    for info in tile_info:
        is_valid = True

        # VÃ©rifier le fichier DSM
        if not os.path.isfile(info['dsm_path']):
            missing += 1
            is_valid = False
            continue

        # VÃ©rifier les fichiers de masque
        for threshold, mask_path in info['mask_paths'].items():
            if not os.path.isfile(mask_path):
                missing += 1
                is_valid = False
                break

        if is_valid:
            valid_info.append(info)

    if missing > 0:
        print(f"Attention: {missing}/{total} tuiles manquantes ou corrompues.")
        print(f"Nombre de tuiles valides: {len(valid_info)}/{total}")

    return len(valid_info) == total, valid_info, total

def create_dataset_from_site(dsm_path, mask_paths, thresholds, tile_size, overlap=0.0,
                             output_dir=None, site_prefix=None, min_valid_ratio=0.7,
                             force_regenerate=False):
    """
    CrÃ©e un ensemble de tuiles Ã  partir d'un site (fichier DSM et masques).
    Sauvegarde les tuiles sur disque si un rÃ©pertoire de sortie est spÃ©cifiÃ©.

    Args:
        dsm_path: Chemin vers le fichier DSM
        mask_paths: Dictionnaire des chemins vers les masques par seuil
        thresholds: Liste des seuils de hauteur
        tile_size: Taille des tuiles en pixels
        overlap: Chevauchement entre les tuiles (0.0 = pas de chevauchement, 0.5 = 50% de chevauchement)
        output_dir: RÃ©pertoire de sortie pour les tuiles (ou None pour ne pas sauvegarder)
        site_prefix: PrÃ©fixe du site pour nommer les fichiers
        min_valid_ratio: Ratio minimum de pixels valides pour conserver une tuile
        force_regenerate: Si True, rÃ©gÃ©nÃ¨re toutes les tuiles mÃªme si elles existent dÃ©jÃ 

    Returns:
        Liste d'informations sur les tuiles crÃ©Ã©es
    """
    # Ouvrir le fichier DSM pour accÃ©der aux mÃ©tadonnÃ©es
    with rasterio.open(dsm_path) as src:
        width = src.width
        height = src.height

    # Calculer le pas entre les tuiles (avec chevauchement)
    stride = int(tile_size * (1 - overlap))

    # Calculer les positions de dÃ©part des tuiles
    row_starts = list(range(0, height - tile_size + 1, stride))
    col_starts = list(range(0, width - tile_size + 1, stride))

    # Assurer qu'il y a au moins une tuile mÃªme pour les petites images
    if not row_starts:
        row_starts = [0]
    if not col_starts:
        col_starts = [0]

    # CrÃ©er le rÃ©pertoire de sortie si nÃ©cessaire
    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)

    # CrÃ©er un fichier de verrouillage pour indiquer que le traitement est en cours
    if output_dir is not None:
        lock_file = os.path.join(output_dir, f"{site_prefix}_processing.lock")
        with open(lock_file, 'w') as f:
            f.write(f"Processing started at: {datetime.now().isoformat()}\n")

    try:
        # Liste pour stocker les informations sur les tuiles
        tile_info = []

        # Compteurs pour le suivi
        total_tiles = len(row_starts) * len(col_starts)
        valid_tiles = 0

        print(f"Extraction des tuiles pour {site_prefix} (taille: {tile_size}, chevauchement: {overlap*100:.0f}%)...")

        # Parcourir toutes les positions de dÃ©part
        for row_idx, row_start in enumerate(row_starts):
            for col_idx, col_start in enumerate(col_starts):
                # DÃ©finir la fenÃªtre pour cette tuile
                window = Window(col_start, row_start, tile_size, tile_size)

                # Chemins des fichiers de tuile
                if output_dir is not None:
                    dsm_tile_path = os.path.join(output_dir, f"{site_prefix}_r{row_idx}_c{col_idx}_dsm.npy")
                    mask_tile_paths = {
                        threshold: os.path.join(output_dir, f"{site_prefix}_r{row_idx}_c{col_idx}_mask_{threshold}m.npy")
                        for threshold in thresholds
                    }

                    # VÃ©rifier si toutes les tuiles existent et sont valides
                    all_tiles_exist = not force_regenerate and os.path.isfile(dsm_tile_path)

                    if all_tiles_exist:
                        for threshold, mask_path in mask_tile_paths.items():
                            if not os.path.isfile(mask_path):
                                all_tiles_exist = False
                                break

                        # VÃ©rification supplÃ©mentaire de l'intÃ©gritÃ© des fichiers
                        if all_tiles_exist:
                            try:
                                # Essayer de charger un Ã©chantillon pour vÃ©rifier l'intÃ©gritÃ©
                                dsm_sample = np.load(dsm_tile_path)
                                if dsm_sample.shape != (tile_size, tile_size):
                                    all_tiles_exist = False
                                else:
                                    # VÃ©rifier un masque au hasard
                                    test_threshold = random.choice(thresholds)
                                    mask_sample = np.load(mask_tile_paths[test_threshold])
                                    if mask_sample.shape != (tile_size, tile_size):
                                        all_tiles_exist = False
                            except Exception:
                                # Si le chargement Ã©choue, le fichier est probablement corrompu
                                all_tiles_exist = False
                                # Supprimer les fichiers corrompus
                                if os.path.exists(dsm_tile_path):
                                    os.remove(dsm_tile_path)
                                for path in mask_tile_paths.values():
                                    if os.path.exists(path):
                                        os.remove(path)

                    if all_tiles_exist:
                        # Ajouter les informations sur cette tuile existante
                        tile_info.append({
                            'site': site_prefix,
                            'row_idx': row_idx,
                            'col_idx': col_idx,
                            'window': window,
                            'dsm_path': dsm_tile_path,
                            'mask_paths': mask_tile_paths
                        })
                        valid_tiles += 1
                        continue

                # Si les tuiles n'existent pas encore, les extraire
                # Extraire la tuile DSM
                with rasterio.open(dsm_path) as src:
                    dsm_tile = src.read(1, window=window)

                # VÃ©rifier si la tuile DSM a suffisamment de donnÃ©es valides
                dsm_valid_ratio = np.sum(~np.isnan(dsm_tile)) / (tile_size * tile_size)

                if dsm_valid_ratio >= min_valid_ratio:
                    # Extraire les tuiles des masques aux mÃªmes positions
                    mask_tiles = {}
                    all_masks_valid = True

                    for threshold in thresholds:
                        with rasterio.open(mask_paths[threshold]) as src:
                            mask_tile = src.read(1, window=window)

                        # VÃ©rifier si la tuile de masque a suffisamment de donnÃ©es valides
                        mask_valid_ratio = np.sum(mask_tile != 255) / (tile_size * tile_size)

                        if mask_valid_ratio >= min_valid_ratio:
                            mask_tiles[threshold] = mask_tile
                        else:
                            all_masks_valid = False
                            break

                    # Si toutes les tuiles sont valides, les sauvegarder et ajouter aux informations
                    if all_masks_valid:
                        if output_dir is not None:
                            # Sauvegarder la tuile DSM
                            dsm_tile_path = os.path.join(output_dir, f"{site_prefix}_r{row_idx}_c{col_idx}_dsm.npy")
                            np.save(dsm_tile_path, dsm_tile)

                            # Sauvegarder les tuiles des masques
                            mask_tile_paths = {}
                            for threshold in thresholds:
                                mask_tile_path = os.path.join(
                                    output_dir,
                                    f"{site_prefix}_r{row_idx}_c{col_idx}_mask_{threshold}m.npy"
                                )
                                np.save(mask_tile_path, mask_tiles[threshold])
                                mask_tile_paths[threshold] = mask_tile_path
                        else:
                            # Si pas de sauvegarde, utiliser des chemins None
                            dsm_tile_path = None
                            mask_tile_paths = {threshold: None for threshold in thresholds}

                        # Ajouter les informations sur cette tuile
                        tile_info.append({
                            'site': site_prefix,
                            'row_idx': row_idx,
                            'col_idx': col_idx,
                            'window': window,
                            'dsm_path': dsm_tile_path,
                            'mask_paths': mask_tile_paths
                        })
                        valid_tiles += 1

        print(f"Extraction terminÃ©e: {valid_tiles}/{total_tiles} tuiles valides ({valid_tiles/total_tiles*100:.1f}%)")

        # Supprimer le fichier de verrouillage une fois le traitement terminÃ©
        if output_dir is not None and os.path.exists(lock_file):
            os.remove(lock_file)

        return tile_info

    except Exception as e:
        # En cas d'erreur, enregistrer l'information et supprimer le verrou
        if output_dir is not None:
            error_log = os.path.join(output_dir, f"{site_prefix}_error.log")
            with open(error_log, 'w') as f:
                f.write(f"Error at: {datetime.now().isoformat()}\n")
                f.write(f"Error: {str(e)}\n")

            if os.path.exists(lock_file):
                os.remove(lock_file)

        raise e  # Relancer l'exception pour la gestion en amont



def check_for_interrupted_processing(tiles_dir):
    """
    VÃ©rifie s'il existe des fichiers de verrouillage indiquant un traitement interrompu.

    Args:
        tiles_dir: RÃ©pertoire contenant les tuiles

    Returns:
        Liste des prÃ©fixes de sites avec des traitements interrompus
    """
    lock_files = glob.glob(os.path.join(tiles_dir, '*_processing.lock'))
    interrupted_sites = []

    for lock_file in lock_files:
        site_prefix = os.path.basename(lock_file).replace('_processing.lock', '')

        # VÃ©rifier l'Ã¢ge du fichier de verrouillage
        mod_time = os.path.getmtime(lock_file)
        age_hours = (time.time() - mod_time) / 3600

        if age_hours > 12:  # Si le fichier a plus de 12 heures
            interrupted_sites.append(site_prefix)
            print(f"DÃ©tectÃ© traitement interrompu pour {site_prefix} (il y a {age_hours:.1f} heures)")

    return interrupted_sites

def clean_incomplete_tiles(tiles_dir, site_prefix):
    """
    Nettoie les tuiles incomplÃ¨tes pour un site spÃ©cifique.

    Args:
        tiles_dir: RÃ©pertoire contenant les tuiles
        site_prefix: PrÃ©fixe du site
    """
    pattern = os.path.join(tiles_dir, f"{site_prefix}_r*_c*_*.npy")
    files = glob.glob(pattern)

    # Grouper les fichiers par tuile (position row, col)
    tile_groups = {}
    for file_path in files:
        base_name = os.path.basename(file_path)
        # Extraire r{row_idx}_c{col_idx}
        match = re.search(r'(r\d+_c\d+)', base_name)
        if match:
            key = match.group(1)
            if key not in tile_groups:
                tile_groups[key] = []
            tile_groups[key].append(file_path)

    # Pour chaque tuile, vÃ©rifier si tous les fichiers existent
    for position, tile_files in tile_groups.items():
        # Une tuile complÃ¨te a 1 + len(thresholds) fichiers
        # (1 DSM + 1 masque par seuil)
        if len(tile_files) != 1 + len(config.THRESHOLDS):
            print(f"Suppression de tuile incomplÃ¨te: {position}")
            for file_path in tile_files:
                os.remove(file_path)

def process_external_test_data(data_pairs, config):
    """
    Traite les donnÃ©es de test externes.

    Args:
        data_pairs: Liste des tuples (chm_path, dsm_path, prefix) pour les donnÃ©es externes
        config: Configuration du projet

    Returns:
        Liste d'informations sur les tuiles de test externes
    """
    if not data_pairs:
        print("Aucune paire de donnÃ©es externe Ã  traiter.")
        return []

    print("\nTraitement des donnÃ©es de test externes:")
    for i, (chm_path, dsm_path, prefix) in enumerate(data_pairs):
        print(f"Paire {i+1}: {prefix}")

    # CrÃ©er un rÃ©pertoire pour les masques externes
    external_masks_dir = os.path.join(config.PROCESSED_DIR, 'external_gap_masks')
    os.makedirs(external_masks_dir, exist_ok=True)

    # CrÃ©er un rÃ©pertoire pour les tuiles externes
    external_tiles_dir = os.path.join(config.PROCESSED_DIR, 'external_tiles')
    os.makedirs(external_tiles_dir, exist_ok=True)

    # Traiter chaque paire externe
    external_tile_info = []

    for chm_path, dsm_path, prefix in data_pairs:
        # CrÃ©er les masques de trouÃ©es
        mask_paths = {
            threshold: os.path.join(external_masks_dir, f'{prefix}_gap_mask_{threshold}m.tif')
            for threshold in config.THRESHOLDS
        }

        # VÃ©rifier si les masques doivent Ãªtre crÃ©Ã©s
        if not all(os.path.exists(path) for path in mask_paths.values()):
            print(f"\nCrÃ©ation des masques de trouÃ©es pour le site externe {prefix}...")
            create_gap_masks(chm_path, config.THRESHOLDS, external_masks_dir, prefix)

        # Extraire les tuiles
        site_output_dir = os.path.join(external_tiles_dir, prefix)
        os.makedirs(site_output_dir, exist_ok=True)

        site_info = create_dataset_from_site(
            dsm_path, mask_paths, config.THRESHOLDS, config.TILE_SIZE,
            overlap=config.OVERLAP, output_dir=site_output_dir, site_prefix=prefix
        )

        external_tile_info.extend(site_info)

    # Sauvegarder les informations sur les tuiles externes
    external_info_path = os.path.join(config.PROCESSED_DIR, 'external_tile_info.pkl')
    with open(external_info_path, 'wb') as f:
        pickle.dump(external_tile_info, f)

    print(f"\nTraitement des donnÃ©es externes terminÃ©: {len(external_tile_info)} tuiles crÃ©Ã©es.")
    return external_tile_info

class GapDataGenerator(Sequence):
    """
    GÃ©nÃ©rateur de donnÃ©es pour l'entraÃ®nement des modÃ¨les.
    Prend en entrÃ©e les informations sur les tuiles et gÃ©nÃ¨re des batchs
    avec les entrÃ©es (DSM, seuil) et les sorties (masque).
    """
    def __init__(self, tile_info, thresholds, batch_size=32, shuffle=True, augment=False):
        """
        Initialise le gÃ©nÃ©rateur de donnÃ©es.

        Args:
            tile_info: Liste d'informations sur les tuiles
            thresholds: Liste des seuils de hauteur
            batch_size: Taille des batchs
            shuffle: Si True, mÃ©lange les donnÃ©es Ã  chaque Ã©poque
            augment: Si True, applique des augmentations de donnÃ©es Ã  la volÃ©e
        """
        self.tile_info = tile_info
        self.thresholds = thresholds
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment

        # CrÃ©er une liste des indices et des seuils associÃ©s
        self.indices = []
        for i, info in enumerate(tile_info):
            for threshold in thresholds:
                self.indices.append((i, threshold))

        # MÃ©langer les indices si demandÃ©
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        """Retourne le nombre de batchs par Ã©poque"""
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        """
        GÃ©nÃ¨re un batch de donnÃ©es.

        Args:
            idx: Indice du batch

        Returns:
            Tuple ([dsm_batch, threshold_batch], mask_batch)
        """
        # Indices pour ce batch
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_size = len(batch_indices)

        # DÃ©terminer la taille des tuiles Ã  partir de la premiÃ¨re tuile
        first_info = self.tile_info[batch_indices[0][0]]
        if first_info['dsm_path'] is not None:
            dsm_tile = np.load(first_info['dsm_path'])
            tile_size = dsm_tile.shape[0]
        else:
            # Valeur par dÃ©faut si les chemins sont None
            tile_size = 256

        # Initialiser les arrays pour le batch
        dsm_batch = np.zeros((batch_size, tile_size, tile_size, 1), dtype=np.float32)
        threshold_batch = np.zeros((batch_size, 1), dtype=np.float32)
        mask_batch = np.zeros((batch_size, tile_size, tile_size, 1), dtype=np.float32)

        # Remplir le batch
        for i, (info_idx, threshold) in enumerate(batch_indices):
            info = self.tile_info[info_idx]

            # Charger la tuile DSM
            if info['dsm_path'] is not None:
                dsm_tile = np.load(info['dsm_path'])
            else:
                # Cas oÃ¹ les tuiles sont passÃ©es directement (sans sauvegarde)
                continue

            # Charger la tuile de masque
            if info['mask_paths'][threshold] is not None:
                mask_tile = np.load(info['mask_paths'][threshold])
            else:
                continue

            # Normaliser la tuile DSM
            dsm_valid = ~np.isnan(dsm_tile)
            if np.any(dsm_valid):
                dsm_min = np.nanmin(dsm_tile)
                dsm_max = np.nanmax(dsm_tile)
                dsm_range = dsm_max - dsm_min
                if dsm_range > 0:
                    dsm_tile = np.where(dsm_valid, (dsm_tile - dsm_min) / dsm_range, 0)
                else:
                    dsm_tile = np.where(dsm_valid, 0, 0)

            # PrÃ©parer le masque (1=trouÃ©e, 0=non-trouÃ©e, 255=nodata)
            mask_valid = (mask_tile != 255)
            mask_binary = np.where(mask_valid, (mask_tile > 0).astype(np.float32), 0)

            # Appliquer des augmentations si demandÃ©
            if self.augment:
                # Rotation alÃ©atoire (0, 90, 180 ou 270 degrÃ©s)
                k = np.random.randint(0, 4)
                if k > 0:
                    dsm_tile = np.rot90(dsm_tile, k=k)
                    mask_binary = np.rot90(mask_binary, k=k)

                # Miroir horizontal alÃ©atoire
                if np.random.random() < 0.5:
                    dsm_tile = np.fliplr(dsm_tile)
                    mask_binary = np.fliplr(mask_binary)

                # Miroir vertical alÃ©atoire
                if np.random.random() < 0.5:
                    dsm_tile = np.flipud(dsm_tile)
                    mask_binary = np.flipud(mask_binary)

            # Ajouter au batch
            dsm_batch[i, :, :, 0] = dsm_tile
            threshold_batch[i, 0] = threshold / max(self.thresholds)  # Normaliser entre 0 et 1
            mask_batch[i, :, :, 0] = mask_binary

        return [dsm_batch, threshold_batch], mask_batch

    def on_epoch_end(self):
        """AppelÃ© Ã  la fin de chaque Ã©poque"""
        if self.shuffle:
            np.random.shuffle(self.indices)

def create_balanced_indices(tile_info, thresholds, gap_ratios):
    """
    CrÃ©e une liste d'indices Ã©quilibrÃ©e basÃ©e sur les proportions de trouÃ©es.

    Args:
        tile_info: Liste d'informations sur les tuiles
        thresholds: Liste des seuils de hauteur
        gap_ratios: Proportions de trouÃ©es par seuil

    Returns:
        Liste d'indices (tuile_idx, seuil) rÃ©pÃ©tÃ©s selon leur importance
    """
    # Calculer les poids d'Ã©chantillonnage
    weight_multiplier = {}
    for threshold in thresholds:
        ratio = gap_ratios.get(threshold, 0.5)  # Valeur par dÃ©faut si non disponible

        # Calculer le facteur de multiplication
        if ratio > 0.5:
            # Si plus de 50% des pixels sont des trouÃ©es, donner plus de poids aux non-trouÃ©es
            weight_multiplier[threshold] = ratio / (1 - ratio + 1e-6)
        else:
            # Sinon, donner plus de poids aux trouÃ©es
            weight_multiplier[threshold] = (1 - ratio) / (ratio + 1e-6)

        # Limiter les poids extrÃªmes
        weight_multiplier[threshold] = min(weight_multiplier[threshold], 5.0)

    # CrÃ©er des groupes d'indices par seuil
    indices_by_threshold = {}
    for threshold in thresholds:
        indices_by_threshold[threshold] = []

    # Classer chaque tuile selon son seuil
    for i, info in enumerate(tile_info):
        for threshold in thresholds:
            indices_by_threshold[threshold].append((i, threshold))

    # CrÃ©er une liste Ã©quilibrÃ©e en rÃ©pÃ©tant les classes sous-reprÃ©sentÃ©es
    balanced_indices = []
    for threshold, indices in indices_by_threshold.items():
        # DÃ©terminer le nombre de rÃ©pÃ©titions selon le poids
        # Convertir en nombre entier avec un minimum de 1
        repetitions = max(1, int(round(weight_multiplier[threshold])))
        balanced_indices.extend(indices * repetitions)

    # MÃ©langer les indices
    np.random.seed(42)  # Pour reproductibilitÃ©
    np.random.shuffle(balanced_indices)

    # Retourner les indices Ã©quilibrÃ©s
    return balanced_indices

def update_balanced_indices(existing_indices, new_tile_info, old_tile_info, thresholds, gap_ratios):
    """
    Met Ã  jour les indices Ã©quilibrÃ©s avec de nouvelles tuiles.

    Args:
        existing_indices: Liste d'indices Ã©quilibrÃ©s existante
        new_tile_info: Liste des nouvelles informations de tuiles
        old_tile_info: Liste des anciennes informations de tuiles
        thresholds: Liste des seuils de hauteur
        gap_ratios: Proportions de trouÃ©es par seuil

    Returns:
        Liste mise Ã  jour d'indices Ã©quilibrÃ©s
    """
    # Si pas de nouvelles tuiles, retourner les indices existants
    if not new_tile_info:
        return existing_indices

    # DÃ©calage d'index pour les nouvelles tuiles
    offset = len(old_tile_info)

    print(f"Mise Ã  jour des indices Ã©quilibrÃ©s avec {len(new_tile_info)} nouvelles tuiles")
    print(f"Offset pour les nouveaux indices: {offset}")

    # CrÃ©er des indices pour les nouvelles tuiles uniquement
    new_indices = []
    for i in range(len(new_tile_info)):
        for threshold in thresholds:
            new_indices.append((i + offset, threshold))

    # Calculer les poids d'Ã©quilibrage
    weight_multiplier = {}
    for threshold in thresholds:
        ratio = gap_ratios.get(threshold, 0.5)
        if ratio > 0.5:
            weight_multiplier[threshold] = ratio / (1 - ratio + 1e-6)
        else:
            weight_multiplier[threshold] = (1 - ratio) / (ratio + 1e-6)
        weight_multiplier[threshold] = min(weight_multiplier[threshold], 5.0)

    # Grouper les nouveaux indices par seuil
    new_indices_by_threshold = {threshold: [] for threshold in thresholds}
    for idx, threshold in new_indices:
        new_indices_by_threshold[threshold].append((idx, threshold))

    # Ã‰quilibrer les nouveaux indices
    balanced_new_indices = []
    for threshold, indices in new_indices_by_threshold.items():
        repetitions = max(1, int(round(weight_multiplier[threshold])))
        print(f"  Seuil {threshold}m: {len(indices)} tuiles x {repetitions} rÃ©pÃ©titions = {len(indices) * repetitions} indices")
        balanced_new_indices.extend(indices * repetitions)

    # MÃ©langer les nouveaux indices
    np.random.seed(42)
    np.random.shuffle(balanced_new_indices)

    # Combiner avec les indices existants
    combined_indices = existing_indices + balanced_new_indices

    # Re-mÃ©langer l'ensemble
    np.random.shuffle(combined_indices)

    return combined_indices


# PARTIE 2 : EXÃ‰CUTION

In [12]:
# =====================================================================
# SECTION 4: PRÃ‰PARATION DES DONNÃ‰ES
# =====================================================================

def main():
    """Fonction principale pour la prÃ©paration des donnÃ©es"""
    print("=== PRÃ‰PARATION DES DONNÃ‰ES POUR LA DÃ‰TECTION DES TROUÃ‰ES FORESTIÃˆRES ===\n")

    # Sauvegarder la configuration
    config.save_config()

    # Trouver les paires CHM/DSM dans le dossier de donnÃ©es principal
    print("\n1. RECHERCHE DES PAIRES CHM/DSM")
    print("--------------------------------")
    data_pairs = find_chm_dsm_pairs(config.DATA_DIR)
    print(f"Nombre de paires CHM/DSM trouvÃ©es dans le dossier de donnÃ©es: {len(data_pairs)}")

    for i, (chm_path, dsm_path, prefix) in enumerate(data_pairs):
        print(f"Paire {i+1}: {prefix}")
        print(f"  CHM: {os.path.basename(chm_path)}")
        print(f"  DSM: {os.path.basename(dsm_path)}")

    # Trouver les paires CHM/DSM dans le dossier de test externe
    external_test_pairs = find_chm_dsm_pairs(config.DATA_EXTERNAL_TEST_DIR)
    print(f"\nNombre de paires CHM/DSM trouvÃ©es dans le dossier de test externe: {len(external_test_pairs)}")

    for i, (chm_path, dsm_path, prefix) in enumerate(external_test_pairs):
        print(f"Paire {i+1}: {prefix}")
        print(f"  CHM: {os.path.basename(chm_path)}")
        print(f"  DSM: {os.path.basename(dsm_path)}")

    # VÃ©rifier que les paires de donnÃ©es existent
    if len(data_pairs) == 0:
        print("\nATTENTION: Aucune paire CHM/DSM trouvÃ©e dans le dossier de donnÃ©es!")
        print("Veuillez placer vos fichiers avec suffixes *CHM.tif et *DSM.tif dans le dossier:")
        print(config.DATA_DIR)
        return

    # Analyse et traitement robuste des paires
    print("\n2. ANALYSE ET TRAITEMENT DES PAIRES CHM/DSM")
    print("------------------------------------------")

    # CrÃ©er une liste pour stocker les paires traitÃ©es
    processed_pairs = []

    for chm_path, dsm_path, prefix in data_pairs:
        # Traiter la paire de maniÃ¨re robuste
        dsm_processed, chm_processed, report, success = process_raster_pair_robustly(
            dsm_path, chm_path, prefix, config
        )

        if success:
            processed_pairs.append((chm_processed, dsm_processed, prefix))

    print(f"\nNombre de paires traitÃ©es avec succÃ¨s: {len(processed_pairs)}/{len(data_pairs)}")

    # Traiter les paires externes sÃ©parÃ©ment
    processed_external_pairs = []

    if external_test_pairs:
        print("\nTraitement des paires de test externes:")
        for chm_path, dsm_path, prefix in external_test_pairs:
            dsm_processed, chm_processed, report, success = process_raster_pair_robustly(
                dsm_path, chm_path, prefix, config
            )

            if success:
                processed_external_pairs.append((chm_processed, dsm_processed, prefix))

        print(f"Nombre de paires externes traitÃ©es avec succÃ¨s: {len(processed_external_pairs)}/{len(external_test_pairs)}")

    #  CrÃ©ation des masques de trouÃ©es
    print("\n3. CRÃ‰ATION DES MASQUES DE TROUÃ‰ES")
    print("----------------------------------")

    gap_masks_dir = os.path.join(config.PROCESSED_DIR, 'gap_masks')
    gap_masks_by_prefix = {}

    for chm_path, dsm_path, prefix in processed_pairs:
        mask_paths = create_gap_masks(chm_path, config.THRESHOLDS, gap_masks_dir, prefix)
        gap_masks_by_prefix[prefix] = mask_paths

    # Calcul ou chargement des statistiques globales de trouÃ©es
    gap_stats_path = os.path.join(config.PROCESSED_DIR, 'gap_statistics.json')
    force_recalculate_stats = False  # DÃ©finir Ã  True pour forcer le recalcul

    # VÃ©rifier si des nouveaux masques ont Ã©tÃ© crÃ©Ã©s dans cette session
    new_prefixes = set(gap_masks_by_prefix.keys())
    old_prefixes = set()

    # Charger les statistiques prÃ©cÃ©dentes pour comparer les prÃ©fixes
    if os.path.exists(gap_stats_path) and not force_recalculate_stats:
        gap_stats = safe_load_json(gap_stats_path)
        if gap_stats and 'prefixes' in gap_stats:
            old_prefixes = set(gap_stats['prefixes'])

    # Si de nouveaux prÃ©fixes sont prÃ©sents, recalculer les statistiques
    new_prefixes_detected = not old_prefixes.issuperset(new_prefixes)

    if new_prefixes_detected:
        print(f"Nouveaux prÃ©fixes dÃ©tectÃ©s: {new_prefixes - old_prefixes}")
        print("Recalcul des statistiques pour inclure les nouveaux sites...")
        gap_ratios = calculate_and_save_gap_statistics(gap_masks_by_prefix, config)
    elif os.path.exists(gap_stats_path) and not force_recalculate_stats:
        gap_stats = safe_load_json(gap_stats_path)

        if gap_stats and 'gap_ratios' in gap_stats:
            print("\nStatistiques globales des trouÃ©es (chargÃ©es):")
            for threshold_str, ratio in gap_stats['gap_ratios'].items():
                threshold = int(threshold_str)  # Convertir la clÃ© string en int
                print(f"  Seuil {threshold}m: {float(ratio):.2%} de pixels sont des trouÃ©es")
            gap_ratios = {int(k): float(v) for k, v in gap_stats['gap_ratios'].items()}
        else:
            print("Fichier de statistiques incomplet ou invalide. Recalcul...")
            gap_ratios = calculate_and_save_gap_statistics(gap_masks_by_prefix, config)
    else:
        print("\nCalcul des statistiques globales des trouÃ©es...")
        gap_ratios = calculate_and_save_gap_statistics(gap_masks_by_prefix, config)

    # Sauvegarder les ratios pour utilisation lors de l'entraÃ®nement
    # Cette information sera utilisÃ©e par le sampler Ã©quilibrÃ©
    gap_ratios_path = os.path.join(config.PROCESSED_DIR, 'gap_ratios.json')
    with open(gap_ratios_path, 'w') as f:
        json.dump({str(k): float(v) for k, v in gap_ratios.items()}, f, indent=2)

    # Division des sites
    print("\n4. DIVISION DES SITES")
    print("--------------------")

    # VÃ©rifier si la division des sites a dÃ©jÃ  Ã©tÃ© effectuÃ©e
    division_info_path = os.path.join(config.PROCESSED_DIR, 'site_split.pkl')

    if os.path.isfile(division_info_path):
        print("Chargement de la division existante des sites...")
        with open(division_info_path, 'rb') as f:
            site_split = pickle.load(f)

        # Mettre Ã  jour la division avec les nouveaux sites
        site_split = update_site_split(
            site_split,
            processed_pairs,
            train_percent=1-(config.TEST_SPLIT+config.VAL_SPLIT),
            val_percent=config.VAL_SPLIT,
            test_percent=config.TEST_SPLIT
        )
    else:
        print("Division des sites en ensembles d'entraÃ®nement, validation et test...")
        site_split = split_sites_by_percentage(
            processed_pairs,
            train_percent=1-(config.TEST_SPLIT+config.VAL_SPLIT),
            val_percent=config.VAL_SPLIT,
            test_percent=config.TEST_SPLIT
        )

    # Sauvegarder la division mise Ã  jour
    with open(division_info_path, 'wb') as f:
        pickle.dump(site_split, f)

    print(f"\nRÃ©partition des sites:")
    print(f"Sites d'entraÃ®nement ({len(site_split['train'])}):", [p[2] for p in site_split['train']])
    print(f"Sites de validation ({len(site_split['val'])}):", [p[2] for p in site_split['val']])
    print(f"Sites de test ({len(site_split['test'])}):", [p[2] for p in site_split['test']])

    # Extraction des tuiles
    print("\n5. EXTRACTION DES TUILES")
    print("----------------------")

    # VÃ©rifier les traitements interrompus
    print("\nVÃ©rification des traitements interrompus...")
    interrupted_sites = check_for_interrupted_processing(config.TRAIN_TILES_DIR)
    interrupted_sites.extend(check_for_interrupted_processing(config.VAL_TILES_DIR))
    interrupted_sites.extend(check_for_interrupted_processing(config.TEST_TILES_DIR))

    # Nettoyer les tuiles incomplÃ¨tes si nÃ©cessaire
    if interrupted_sites:
        print(f"Nettoyage des tuiles incomplÃ¨tes pour {len(interrupted_sites)} sites...")
        for site in interrupted_sites:
            # DÃ©terminer dans quel ensemble se trouve le site
            for subset in ['train', 'val', 'test']:
                for _, _, prefix in site_split[subset]:
                    if prefix == site:
                        tiles_dir = getattr(config, f"{subset.upper()}_TILES_DIR")
                        clean_incomplete_tiles(tiles_dir, site)
                        break

    # VÃ©rifier si les informations de tuiles existent dÃ©jÃ 
    train_info_path = os.path.join(config.TILES_DIR, 'train_tile_info.pkl')
    val_info_path = os.path.join(config.TILES_DIR, 'val_tile_info.pkl')
    test_info_path = os.path.join(config.TILES_DIR, 'test_tile_info.pkl')

    # Traiter les tuiles d'entraÃ®nement
    print("\nVÃ©rification des tuiles d'entraÃ®nement...")
    if os.path.isfile(train_info_path):
        with open(train_info_path, 'rb') as f:
            train_tile_info = pickle.load(f)

        # VÃ©rifier l'intÃ©gritÃ© des tuiles existantes
        is_valid, valid_info, total = verify_tiles_integrity(train_tile_info)

        if is_valid:
            print(f"Chargement rÃ©ussi: {len(valid_info)} tuiles valides.")
        else:
            print(f"Certaines tuiles sont manquantes, conservation de {len(valid_info)}/{total} tuiles valides.")
            train_tile_info = valid_info
    else:
        print("Aucune information sur les tuiles d'entraÃ®nement existante.")
        train_tile_info = []

    # Mettre Ã  jour les tuiles d'entraÃ®nement avec les nouveaux sites
    train_tile_info = update_tile_info(
        train_tile_info, site_split, 'train', config.TRAIN_TILES_DIR, config
    )

    # Sauvegarder les informations mises Ã  jour
    with open(train_info_path, 'wb') as f:
        pickle.dump(train_tile_info, f)

    # Traiter les tuiles de validation
    print("\nVÃ©rification des tuiles de validation...")
    if os.path.isfile(val_info_path):
        with open(val_info_path, 'rb') as f:
            val_tile_info = pickle.load(f)

        # VÃ©rifier l'intÃ©gritÃ© des tuiles existantes
        is_valid, valid_info, total = verify_tiles_integrity(val_tile_info)

        if is_valid:
            print(f"Chargement rÃ©ussi: {len(valid_info)} tuiles valides.")
        else:
            print(f"Certaines tuiles sont manquantes, conservation de {len(valid_info)}/{total} tuiles valides.")
            val_tile_info = valid_info
    else:
        print("Aucune information sur les tuiles de validation existante.")
        val_tile_info = []

    # Mettre Ã  jour les tuiles de validation avec les nouveaux sites
    val_tile_info = update_tile_info(
        val_tile_info, site_split, 'val', config.VAL_TILES_DIR, config
    )

    # Sauvegarder les informations mises Ã  jour
    with open(val_info_path, 'wb') as f:
        pickle.dump(val_tile_info, f)

    # Traiter les tuiles de test
    print("\nVÃ©rification des tuiles de test...")
    if os.path.isfile(test_info_path):
        with open(test_info_path, 'rb') as f:
            test_tile_info = pickle.load(f)

        # VÃ©rifier l'intÃ©gritÃ© des tuiles existantes
        is_valid, valid_info, total = verify_tiles_integrity(test_tile_info)

        if is_valid:
            print(f"Chargement rÃ©ussi: {len(valid_info)} tuiles valides.")
        else:
            print(f"Certaines tuiles sont manquantes, conservation de {len(valid_info)}/{total} tuiles valides.")
            test_tile_info = valid_info
    else:
        print("Aucune information sur les tuiles de test existante.")
        test_tile_info = []

    # Mettre Ã  jour les tuiles de test avec les nouveaux sites
    test_tile_info = update_tile_info(
        test_tile_info, site_split, 'test', config.TEST_TILES_DIR, config
    )

    # Sauvegarder les informations mises Ã  jour
    with open(test_info_path, 'wb') as f:
        pickle.dump(test_tile_info, f)

    # Traitement des donnÃ©es de test externes
    if processed_external_pairs:
        print("\n5b. TRAITEMENT DES DONNÃ‰ES DE TEST EXTERNES")
        print("----------------------------------------")

        external_tile_info_path = os.path.join(config.PROCESSED_DIR, 'external_tile_info.pkl')

        # VÃ©rifier si des informations sur les tuiles externes existent dÃ©jÃ 
        if os.path.isfile(external_tile_info_path):
            with open(external_tile_info_path, 'rb') as f:
                external_tile_info = pickle.load(f)

            # VÃ©rifier l'intÃ©gritÃ© des tuiles existantes
            is_valid, valid_info, total = verify_tiles_integrity(external_tile_info)

            if is_valid:
                print(f"Chargement rÃ©ussi: {len(valid_info)} tuiles externes valides.")
            else:
                print(f"Certaines tuiles externes sont manquantes, conservation de {len(valid_info)}/{total} tuiles valides.")
                external_tile_info = valid_info
        else:
            print("Aucune information sur les tuiles externes existante.")
            external_tile_info = []

        # Traiter chaque paire externe
        new_external_info = []
        external_gap_masks_dir = os.path.join(config.PROCESSED_DIR, 'external_gap_masks')
        os.makedirs(external_gap_masks_dir, exist_ok=True)

        external_tiles_dir = os.path.join(config.PROCESSED_DIR, 'external_tiles')
        os.makedirs(external_tiles_dir, exist_ok=True)

        for chm_path, dsm_path, prefix in processed_external_pairs:
            # VÃ©rifier si le site est dÃ©jÃ  traitÃ©
            if any(info['site'] == prefix for info in external_tile_info):
                print(f"Site externe {prefix} dÃ©jÃ  traitÃ©. IgnorÃ©.")
                continue

            # CrÃ©er les masques pour ce site externe
            mask_paths = create_gap_masks(chm_path, config.THRESHOLDS, external_gap_masks_dir, prefix)

            # Extraire les tuiles pour ce site
            site_output_dir = os.path.join(external_tiles_dir, prefix)
            os.makedirs(site_output_dir, exist_ok=True)

            site_info = create_dataset_from_site(
                dsm_path, mask_paths, config.THRESHOLDS, config.TILE_SIZE,
                overlap=config.OVERLAP, output_dir=site_output_dir, site_prefix=prefix
            )

            new_external_info.extend(site_info)

        # Mettre Ã  jour les informations sur les tuiles externes
        if new_external_info:
            external_tile_info.extend(new_external_info)

            with open(external_tile_info_path, 'wb') as f:
                pickle.dump(external_tile_info, f)

            print(f"Ajout de {len(new_external_info)} nouvelles tuiles externes, total: {len(external_tile_info)}")

    # CrÃ©er les gÃ©nÃ©rateurs de donnÃ©es
    print("\n6. CRÃ‰ATION DES GÃ‰NÃ‰RATEURS DE DONNÃ‰ES")
    print("-------------------------------------")

    train_generator = GapDataGenerator(train_tile_info, config.THRESHOLDS, batch_size=config.BATCH_SIZE, shuffle=True, augment=True)
    val_generator = GapDataGenerator(val_tile_info, config.THRESHOLDS, batch_size=config.BATCH_SIZE, shuffle=False)
    test_generator = GapDataGenerator(test_tile_info, config.THRESHOLDS, batch_size=config.BATCH_SIZE, shuffle=False)


    print(f"\nStatistiques finales:")
    print(f"  Nombre de tuiles d'entraÃ®nement: {len(train_tile_info)} ({len(train_generator)} batchs de taille {config.BATCH_SIZE})")
    print(f"  Nombre de tuiles de validation: {len(val_tile_info)} ({len(val_generator)} batchs)")
    print(f"  Nombre de tuiles de test: {len(test_tile_info)} ({len(test_generator)} batchs)")
    print(f"  Nombre d'Ã©chantillons d'entraÃ®nement: {len(train_tile_info) * len(config.THRESHOLDS)} (en comptant tous les seuils)")
    print(f"  Augmentation Ã  la volÃ©e activÃ©e pour l'entraÃ®nement")
    print(f"  Total d'Ã©chantillons sur disque: {len(train_tile_info) + len(val_tile_info) + len(test_tile_info)} tuiles")

    print("\n=== PRÃ‰PARATION DES DONNÃ‰ES TERMINÃ‰E ===")

if __name__ == "__main__":
    main()

=== PRÃ‰PARATION DES DONNÃ‰ES POUR LA DÃ‰TECTION DES TROUÃ‰ES FORESTIÃˆRES ===

Configuration sauvegardÃ©e dans /content/drive/MyDrive/ForestGaps_DeepLearning_Workflow/processed/config.json

1. RECHERCHE DES PAIRES CHM/DSM
--------------------------------
Nombre de paires CHM/DSM trouvÃ©es dans le dossier de donnÃ©es: 20
Paire 1: UTM33S_Plot137
  CHM: UTM33S_Plot137_CHM.tif
  DSM: UTM33S_Plot137_DSM.tif
Paire 2: UTM33S_Plot26
  CHM: UTM33S_Plot26_CHM.tif
  DSM: UTM33S_Plot26_DSM.tif
Paire 3: UTM34N_Plot119
  CHM: UTM34N_Plot119_CHM.tif
  DSM: UTM34N_Plot119_DSM.tif
Paire 4: UTM34N_Plot162
  CHM: UTM34N_Plot162_CHM.tif
  DSM: UTM34N_Plot162_DSM.tif
Paire 5: UTM34N_Plot192
  CHM: UTM34N_Plot192_CHM.tif
  DSM: UTM34N_Plot192_DSM.tif
Paire 6: UTM34N_Plot80
  CHM: UTM34N_Plot80_CHM.tif
  DSM: UTM34N_Plot80_DSM.tif
Paire 7: UTM34N_Plot94
  CHM: UTM34N_Plot94_CHM.tif
  DSM: UTM34N_Plot94_DSM.tif
Paire 8: UTM34S_Plot1
  CHM: UTM34S_Plot1_CHM.tif
  DSM: UTM34S_Plot1_DSM.tif
Paire 9: UTM34S_Plot